In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import pandas as pd
import nltk
import numpy as np
import re
import string

KeyboardInterrupt: 

In [ ]:
PATH = 'jigsaw_train_hateoffensive.csv'
df = pd.read_csv(PATH)

In [ ]:
def remove_punct(text):
    new_words = []
    for word in text:
        w = re.sub(r'[^\w\s]','',word) #remove everything except words and space
        w = re.sub(r'_','',w) #how to remove underscore as well
        new_words.append(w)
    return "".join(new_words)

def remove_extras(text):
    text = remove_punct(text)
    text = re.sub("\n+", "", text)
    text = re.sub("\s+$", "", text)
    text = re.sub("\s+", " ", text)
    text = re.sub(f"http\S+", "", text)

    return text

In [ ]:
label_to_num = {
    "neutral": 0,
    "offensive": 1,
    "hate_speech": 2
}

In [ ]:
texts = df["text"].map(remove_extras), 
texts.shape

KeyboardInterrupt: 

In [ ]:
labels = df[df.columns[3:]]
labels = tf.keras.utils.to_categorical(labels["hateoffensive_class"].map(lambda x: label_to_num[x]))


In [ ]:
seq_len = 512
num_samples = len(texts)

Xids = np.zeros((num_samples, seq_len))
Xmask = np.zeros((num_samples, seq_len))
# labels = np.expand_dims(, axis=0).T

Xids.shape, labels.shape

((159571, 512), (1, 159571, 3, 1))

In [ ]:
from transformers import BertTokenizer
from tqdm import tqdm

tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

for i, phrase in enumerate(tqdm(texts)):
    token = tokenizer.encode_plus(
        phrase, max_length=seq_len, add_special_tokens=True, 
        padding="max_length", truncation=True, return_tensors='tf')

    Xids[i, :] = token['input_ids']
    Xmask[i, :] = token['attention_mask']

100%|██████████| 159571/159571 [03:28<00:00, 764.75it/s]


In [ ]:
def map_func(inputs_ids, masks, labels):
    return {
        'input_ids': inputs_ids,
        'attention_mask': masks
    }, labels

In [ ]:
batch_size = 32
dataset = tf.data.Dataset.from_tensor_slices((Xids, Xmask, labels))
dataset = dataset.map(map_func)
dataset = dataset.shuffle(buffer_size=1000).batch(batch_size, drop_remainder=True)
dataset.take(1)

<TakeDataset element_spec=({'input_ids': TensorSpec(shape=(32, 512), dtype=tf.float64, name=None), 'attention_mask': TensorSpec(shape=(32, 512), dtype=tf.float64, name=None)}, TensorSpec(shape=(32, 3), dtype=tf.float32, name=None))>

In [ ]:
split = 0.9
size = int((num_samples/batch_size) * split)

In [ ]:
train_ds = dataset.take(size)
val_ds = dataset.skip(size)

# del [dataset, Xids, Xmask, labels]

In [ ]:
from transformers import TFAutoModel

bert = TFAutoModel.from_pretrained('bert-base-cased')

# make untrainable
bert.trainable = False

Some layers from the model checkpoint at bert-base-cased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [ ]:
input_ids = keras.layers.Input(shape=(seq_len,), name="input_ids", dtype="int32")
attention_mask = keras.layers.Input(shape=(seq_len,), name="attention_mask", dtype="int32")

embeddings = bert.bert(input_ids, attention_mask=attention_mask)[1]

# x = layers.Dense(1024, activation="relu")(embeddings)
# x = layers.Dropout(0.5)(x)
x = layers.Dense(3, activation="softmax")(embeddings)

In [ ]:
model = keras.Model(inputs=[input_ids, attention_mask], outputs=x)

In [ ]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 512)]        0           []                               
                                                                                                  
 attention_mask (InputLayer)    [(None, 512)]        0           []                               
                                                                                                  
 bert (TFBertMainLayer)         TFBaseModelOutputWi  108310272   ['input_ids[0][0]',              
                                thPoolingAndCrossAt               'attention_mask[0][0]']         
                                tentions(last_hidde                                               
                                n_state=(None, 512,                                         

In [ ]:
model.compile(
    optimizer = keras.optimizers.Adam(),
    loss = "categorical_crossentropy",
    metrics=["accuracy"]
)

In [ ]:
import numpy as np
def get_bert_embedding(texts):

    if type(texts) == str:
        texts = [texts]

    num_samples = len(texts)

    Xids = np.zeros((num_samples, seq_len))
    Xmask = np.zeros((num_samples, seq_len))

    for i, phrase in enumerate(texts):
        token = tokenizer.encode_plus(
        phrase, max_length=seq_len, add_special_tokens=True, 
        padding="max_length", truncation=True, return_tensors='tf')

    Xids[i, :] = token['input_ids']
    Xmask[i, :] = token['attention_mask']

    return Xids, Xmask

In [ ]:
ids, mask = get_bert_embedding("There is such a violent earthquake here in my city of Berlin")

In [2]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
### from models.py
from models import *
tokenizer = AutoTokenizer.from_pretrained("Hate-speech-CNERG/bert-base-uncased-hatexplain-rationale-two")
model = Model_Rational_Label.from_pretrained("Hate-speech-CNERG/bert-base-uncased-hatexplain-rationale-two")
inputs = tokenizer('He is a great guy', return_tensors="pt")
prediction_logits, _ = model(input_ids=inputs['input_ids'],attention_mask=inputs['attention_mask'])


Downloading: 100%|██████████| 299/299 [00:00<00:00, 74.7kB/s]
c:\Users\vedan\AppData\Local\Programs\Python\Python38\lib\site-packages\huggingface_hub\file_download.py:127: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\vedan\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Downloading: 100%|██████████| 790/790 [00:00<?, ?B/s] 
Dow

In [4]:
_

tensor([[[ 3.1977, -3.5530],
         [ 3.5791, -4.3285],
         [ 3.5627, -4.4655],
         [ 3.6706, -4.6351],
         [ 3.2578, -4.3120],
         [ 3.3935, -4.2368],
         [ 4.4027, -5.1928]]], grad_fn=<AddBackward0>)